In [ ]:
import requests
import pandas as pd
from datetime import datetime
from datetime import timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import re
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

Khởi tạo một vài tham số

In [ ]:
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(options=options)
#browser = webdriver.Chrome()
current_day = datetime.now()

Lấy dữ liệu vừa thu được ở get_hotel_infomation.ipynb

In [ ]:
hotel_data = pd.read_csv('list_hotel.csv', index_col = 0)
hotel_data.head()

Duyệt danh sách, với mỗi url, dùng selenium để chọn ngày checkin,checkout, lấy thông tin về giá

get_back_next_button() : lấy 2 button chọn tháng của datetime picker

get_first_day_left_table(): lấy ngày đầu tiên của tháng trùng với tháng start_day

choose_day(): chọn start_day và end_day ở datetime picker

choose_number_of_guest(): chọn số lượng khách tối đa (4)

choose_and_submit(): gọi 2 hàm choose_day() và choose_number_of_guest() và submit (Cập nhật giá)

In [ ]:
def get_back_next_button():
    tables = browser.find_elements_by_class_name('t-table-condensed')
    back_button = tables[0].find_element_by_tag_name('i')
    next_button = tables[1].find_element_by_tag_name('i')
    return back_button,next_button
    
def get_first_day_left_table():
    tables = browser.find_elements_by_class_name('t-table-condensed')
    table_left = tables[0]
    month_left_lable = table_left.find_element_by_class_name('t-month')
    tmp_day = month_left_lable.text
    tmp_day = datetime.strptime(tmp_day.replace('Tháng', '1'), '%d %m %Y')
    return tmp_day

def choose_day(start_day, end_day):
    tables = browser.find_elements_by_class_name('t-table-condensed')
    cells_left = tables[0].find_elements_by_tag_name('td')
    cells_right = tables[1].find_elements_by_tag_name('td')
    
    # start day
    for cell in cells_left:
        if cell.text == str(start_day.day):
            cell.click()
            break;
                
    tables = browser.find_elements_by_class_name('t-table-condensed')
    idx = 0
    if end_day.month > start_day.month:
        idx = 1
    
    # end day
    for cell in tables[idx].find_elements_by_tag_name('td'):
        if cell.text == str(end_day.day):
            cell.click()
            break

def choose_number_of_guest():
    browser.find_element_by_class_name('room-guest-select').click()
    select_guest_tag = browser.find_element_by_class_name('guest-select')
    buttons = select_guest_tag.find_elements_by_tag_name('button')
    buttons[1].click()
    buttons[1].click()
    return
            
def choose_and_submit(start_day, end_day, is_choose_number_of_guest):
    browser.find_element_by_tag_name('html').send_keys(Keys.ESCAPE)
    choose_day(start_day, end_day)
    browser.find_element_by_tag_name('html').send_keys(Keys.ESCAPE)
    
    browser.implicitly_wait(3)
    if is_choose_number_of_guest == True:
        choose_number_of_guest()
                
    # submit
    browser.find_element_by_id('search_submit').click()

Lấy dữ liệu sau khi đã cập nhật giá

In [ ]:
def get_data():
    rooms = browser.find_elements_by_class_name('book-choose')
    if len(rooms) == 0:
        return []
    
    result = []
        
    for room in rooms:
        items = room.find_element_by_class_name('product-content').find_elements_by_tag_name('p')
        if len(items) == 5: # nếu đầy đủ thông tin
            size = re.findall('(?<!\.)\d+(?!\.)', items[1].text)[0]
            orientation = items[2].text
            bed = items[3].text.replace(',', ' +')    
            
            other_info = room.find_element_by_class_name('room').find_elements_by_tag_name('tr')
            options_and_price = []
            for info in other_info:
                td_tags = info.find_elements_by_tag_name('td')
                
                # tim số người tối đa
                span_max_guest_tag = td_tags[0].find_element_by_tag_name('span')
                temp_str = span_max_guest_tag.get_attribute('data-original-title')
                max_guest_number = re.findall('(?<!\.)\d+(?!\.)', temp_str)[0]

                have_breakfast = 0
                cancel_ticket = 0
                # tìm danh sách tiện nghi
                p_tags = td_tags[1].find_elements_by_tag_name('p')
                for p_tag in p_tags:
                    if p_tag.text == 'Bao gồm Bữa sáng':
                        have_breakfast = 1
                    if p_tag.text == 'Điều kiện hoàn hủy':
                        cancel_ticket = 1
                
                tmp_tag = browser.find_element_by_css_selector("p[class='price text-lg']")            
                price_tag = tmp_tag.find_element_by_tag_name('strong')
                if price_tag is not None:
                    price = re.findall('(\d+)', price_tag.text.replace(',',''))[0]                
                    options_and_price.append([max_guest_number, have_breakfast, cancel_ticket, price])
            
            result.append([size,orientation,bed,options_and_price])
        
    return result

In [ ]:
def get_attributes_of_hotel():  # Lấy số lượng tiện nghi
    element = browser.find_elements_by_class_name('attribute-value')
    return len(element)

In [ ]:
f = open('{}.csv'.format(current_day.strftime('%d-%m-%Y')), 'w', encoding='utf-8')
f.write('id,checkin_day,size,orientation,bed,attribute_number,max_guest_number,have_breakfast,cancel_ticket,price\n')

In [ ]:
list_idx = hotel_data.index.values.tolist()
for idx in list_idx:
    try:
        url = hotel_data.loc[idx,:]['url']

        browser.get(url)
        browser.find_element_by_tag_name('html').send_keys(Keys.ESCAPE)

        print(hotel_data.loc[idx,:]['name'], end=' : ')

        count = 0
        for i in range(1,8):
            start_day = current_day + timedelta(days=i)
            end_day = current_day + timedelta(days=i+1)

            browser.find_element_by_class_name('t-check-in').click()

            # chuyển cột bên trái về tháng trùng với ngày bắt đầu
            while True:
                back_button,next_button = get_back_next_button()
                tmp_day = get_first_day_left_table()

                if tmp_day.month == start_day.month:
                    choose_and_submit(start_day, end_day, i == 0)
                    break;
                else:
                    if tmp_day.month > start_day.month:
                        back_button.click()
                    else:
                        next_button.click()

            attribute_number = get_attributes_of_hotel()
            records = get_data()
            for r in records:
                checkin_day = start_day.strftime('%d-%m-%Y')
                size = r[0]
                orientation = r[1]
                bed = r[2]
                for o in r[3]:
                    f.write('{},{},{},{},{},{},{},{},{},{}\n'.format(idx,checkin_day,size,orientation,bed,attribute_number, o[0],o[1],o[2],o[3]))
                    print('|', end='')
                    count += 1
    
        print(' ==> found {} record(s)'.format(count))
    except:
        continue

f.close()